In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Load dataset
df = pd.read_csv("lstm_ready_traffic_data.csv")
scaler = MinMaxScaler()
df['scaled_traffic_volume'] = scaler.fit_transform(df[['Total_Traffic_Volume']])
# If 'day_of_week' is text, convert to numeric
if df['day_of_week'].dtype == 'object':
    df['day_of_week'] = df['day_of_week'].astype('category').cat.codes

# Select features and target
features = ['hour', 'day_of_week', 'month', 'year', 'road_name_encoded',
            'location_encoded', 'suburb_encoded', 'average_speed',
            'maximum_speed', 'Speeding_Incident', 'Speeding_Incident_Avg_Speed']
target = 'scaled_traffic_volume'

# Convert features and target to float32 (required by LSTM)
X = df[features].values.astype('float32')
y = df[target].values.astype('float32')

# Create sequences
def create_sequences(X, y, time_steps=24):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, shuffle=False
)


In [ ]:
!pip install keras-tuner --upgrade
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
def build_trimodel(hp):
    model = Sequential()

    # Layer 1
    model.add(LSTM(
        units=hp.Int('lstm_units_1', min_value=32, max_value=128, step=16),
        activation='tanh',
        return_sequences=True,
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    model.add(Dropout(hp.Float('dropout_1', 0.1, 0.4, step=0.1)))

    # Layer 2
    model.add(LSTM(
        units=hp.Int('lstm_units_2', min_value=16, max_value=64, step=16),
        activation='tanh',
        return_sequences=True
    ))
    model.add(Dropout(hp.Float('dropout_2', 0.1, 0.4, step=0.1)))

    # Layer 3
    model.add(LSTM(
        units=hp.Int('lstm_units_3', min_value=8, max_value=32, step=8),
        activation='tanh'
    ))
    model.add(Dropout(hp.Float('dropout_3', 0.1, 0.4, step=0.1)))

    # Output
    model.add(Dense(1))

    model.compile(
        optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
        loss='mse'
    )

    return model

In [ ]:
from keras_tuner.tuners import BayesianOptimization

tuner = BayesianOptimization(
    build_trimodel,
    objective='val_loss',
    max_trials=15,
    executions_per_trial=1,
    directory='traffic_tuner',
    project_name='bayesian_trimodel'
)

tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


Trial 15 Complete [00h 06m 53s]
val_loss: 0.007878963835537434

Best val_loss So Far: 0.006989008281379938
Total elapsed time: 02h 23m 43s


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = best_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 63s 64ms/step - loss: 0.0108 - val_loss: 0.0079
Epoch 2/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 75s 56ms/step - loss: 0.0072 - val_loss: 0.0078
Epoch 3/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 45s 57ms/step - loss: 0.0069 - val_loss: 0.0078
Epoch 4/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 80s 55ms/step - loss: 0.0068 - val_loss: 0.0078
Epoch 5/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - loss: 0.0070 - val_loss: 0.0077
Epoch 6/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 85s 58ms/step - loss: 0.0066 - val_loss: 0.0073
Epoch 7/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 81s 57ms/step - loss: 0.0066 - val_loss: 0.0072
Epoch 8/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 45s 57ms/step - loss: 0.0065 - val_loss: 0.0070
Epoch 9/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 44s 56ms/step - loss: 0.0066 - val_loss: 0.0075
Epoch 10/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 45s 57ms/step - loss: 0.0066 - val_loss: 0.0081
Epoch 11/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 80s 55ms/step - loss: 0.0064 - val_loss: 0.0071
Epoch 12/50
789/789 ━━━━━━━━━━

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

y_pred_bayes = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_bayes))
mae = mean_absolute_error(y_test, y_pred_bayes)
r2 = r2_score(y_test, y_pred_bayes)
mse = mean_squared_error(y_test, y_pred_bayes)

print("\n✅ Bayesian Optimized Trimodel Results:")
print(f"🔹 RMSE: {rmse:.4f}")
print(f"🔹 MAE : {mae:.4f}")
print(f"🔹 R²  : {r2:.4f}")
print(f"🔹 MSE : {mse:.4f}")


395/395 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step

✅ Bayesian Optimized Trimodel Results:
🔹 RMSE: 0.0826
🔹 MAE : 0.0526
🔹 R²  : 0.1213
🔹 MSE : 0.0068


In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(y_test[:700], label='Actual', linewidth=2)
plt.plot(y_pred_bayes[:700], label='Predicted (Bayesian)', linestyle='--', linewidth=2)
plt.title("Bayesian Optimized Trimodel: Actual vs Predicted")
plt.xlabel("Time Step Index")
plt.ylabel("Scaled Traffic Volume")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

np.save("y_bayesian.npy", y_pred_bayes)


In [ ]:
from google.colab import files

files.download("y_bayesian.npy")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>